- competition/dataset : [https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data)
- date : 2021/03/26
- original : [https://www.kaggle.com/jagangupta/stop-the-s-toxic-comments-eda](https://www.kaggle.com/jagangupta/stop-the-s-toxic-comments-eda)

## Stop the S@#$ - Toxic Comments EDA

**✏ 필사 1회** 

### Introduction
인터넷의 익명화가 때로는 사람들이 현실에서는 하지 않을 험악한 말들을 하게 만들 수 있습니다. 우리의 플랫폼에서 증오심을 걸러봅시다.

**Objective:**  
독설 분류를 위한 EDA와 feature-engineering  

**Data Overview:**  
이 대회의 데이터셋은 사람들이 악성에 대해 평가한 위키 말뭉치 데이터셋입니다. 2004~2015년 사이의 유저 페이지와 기사에 관한 토론으로부터 나온 6300만 개의 코멘트들을 포함합니다.  

플랫폼/사이트마다 악성 댓글 검사 절차에 있어 기준이 다를 수 있습니다. 이런 이유로 코멘트들은 다음과 같은 다섯 가지 카테고리로 태그되어 있습니다:  
* toxic (심한)
* severe_toxic (극심한)
* obsence (외설적인)
* threat (위협적인)
* insult (모욕적인)
* identity_hate

이 태깅은 crowdsourcing에 의해 수행되었으며, 이 데이터셋이 여러 사람들에 의해 평가되었고 태깅이 100% 정확도를 가지지 못할 것을 의미합니다. 이 부분에 대해 [이곳](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/46131)에서 논의되고 있습니다.  

In [11]:
# import required packages
# basics
import pandas as pd
import numpy as np

# misc
import gc
import time
import warnings

# stats
from matplotlib.pyplot import imread
from scipy import sparse
import scipy.stats as ss

# viz
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import matplotlib_venn as venn

# nlp
import string
import re
import nltk
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# Tweet tokenizer는 아포스트로피에서 분할x
from nltk.tokenize import TweetTokenizer

# feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# settings
start_time = time.time()
color = sns.color_palette()
sns.set_style('dark')
eng_stopwords = set(stopwords.words('english'))
warnings.filterwarnings('ignore')

lem = WordNetLemmatizer()
tokenizer = TweetTokenizer()

%matplotlib inline

In [12]:
# importing the dataset
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [13]:
# take a peak
train.tail(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [18]:
nrow_train = train.shape[0]
nrow_test = train.shape[0]
sum = nrow_train + nrow_test
print('      : train  : test')
print('rows  :', nrow_train, ':', nrow_test)
print('pers  :', round(nrow_train*100/sum), '    :', round(nrow_test*100/sum))

      : train  : test
rows  : 159571 : 159571
pers  : 50     : 50


이후에 train:test 비율을 30:70으로 나누고 테스트셋 바꿀 수 있습니다.  

트레이닝 셋에서 범주의 불균형에 대해 살펴봅시다.

#### Class Imbalance